In [1]:
import pickle

In [2]:
words = pickle.load(open(f'../dataset/words.pkl', 'rb'))

In [3]:
word2idx = pickle.load(open(f'../dataset/word2idx.pkl', 'rb'))

In [4]:
vectors = pickle.load(open(f'../dataset/vectors.pkl', 'rb'))

In [5]:
train_vocab_list = pickle.load(open(f'../dataset/train_vocab_list.pkl', 'rb'))
train_target_list = pickle.load(open(f'../dataset/train_target_list.pkl', 'rb'))

In [6]:
test_vocab_list = pickle.load(open(f'../dataset/test_vocab_list.pkl', 'rb'))
test_target_list = pickle.load(open(f'../dataset/test_target_list.pkl', 'rb'))

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

In [8]:
def get_weights_matrix(target_vocab):
    target_vocab_list = target_vocab.split(' ')
    matrix_len = len(target_vocab_list)
    weights_matrix = np.zeros((1, 50))
    words_found = 0
    
    for i, word in enumerate(target_vocab_list):        
        try:
            weights_matrix += vectors[word2idx[word]]
            words_found += 1
        except KeyError:
            weights_matrix += np.random.normal(scale=0.6, size=(50,))

    return torch.from_numpy(weights_matrix).unsqueeze(0)

In [17]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        output, hidden = self.rnn(x)
        hidden = hidden.squeeze(0)
        return self.fc(hidden)
    
    def predict(self, x):
        prediction = F.softmax(self.forward(x), dim=1).data[0][0]
        return 1 if prediction > 0.5 else 0

In [18]:
n_epochs = 3
train_len = 67349
test_len = 1821
train_predictions = np.zeros([n_epochs, train_len])
train_losses = np.zeros(n_epochs)
test_predictions = np.zeros([n_epochs, test_len])
test_losses = np.zeros(n_epochs)

model = RNN(50, 300, 2).double()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
for epoch in range(n_epochs):
    for i in range(train_len):
        vocab, target = train_vocab_list[i], train_target_list[i]
        inputs = get_weights_matrix(vocab)
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        train_predictions[epoch][i] = int(predicted)
        loss = criterion(output, torch.tensor([int(target)], dtype=int))
        train_losses[epoch] += loss.item()
        loss.backward()
        optimizer.step()
    for i in range(test_len):
        vocab, target = test_vocab_list[i], test_target_list[i]
        inputs = get_weights_matrix(vocab)
        predicted = model.predict(inputs)
        test_predictions[epoch][i] = int(predicted)
        loss = criterion(output, torch.tensor([int(target)], dtype=int))
        test_losses[epoch] += loss.item()
    print(train_losses[epoch])
    print(test_losses[epoch])

213891.17119042698
8266.784246899695
217163.3567096417
12111.062017529292


In [12]:
pickle.dump(train_predictions, open(f'../dataset/part1_train_predictions.pkl', 'wb'))

In [13]:
pickle.dump(train_losses, open(f'../dataset/part1_train_losses.pkl', 'wb'))

In [14]:
pickle.dump(test_predictions, open(f'../dataset/part1_test_predictions.pkl', 'wb'))

In [15]:
pickle.dump(test_losses, open(f'../dataset/part1_test_losses.pkl', 'wb'))

In [20]:
torch.save(model, '../trained_models/part1_state.chkpt')